In [ ]:
import pandas as pd 
from pandas import json_normalize
import json as js 
import ast as ast
import numpy as np
import re
import gzip

# Creamos objetos con la rutas de los archivos
archivo1 = r'C:/Users/aball/OneDrive/Escritorio/Proyecto 1/Datasets/json/australian_users_items.json' 

archivo2 = r"C:/Users/aball/OneDrive/Escritorio/Proyecto1/Datasets/json/australian_user_reviews.json" 

archivo3 = r"C:/Users/aball/OneDrive/Escritorio/Proyecto 1/Datasets/gz/steam_games.json.gz"

In [ ]:
# Cargamos el archivo json a una lista vacia para convertirlo a DataFrame

lista= []

with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line)) 

# Creamos el DataFrame con la lista Recien creada
df  = pd.DataFrame(lista) 

In [ ]:
# Desanidamos la columna "items" para su posterior uso
df_items = df.explode('items') 

In [ ]:
# Reseteamos el indice para evitar problemas
df_items.reset_index(drop=True, inplace=True) 

In [ ]:
# Normalizamos la columna "items" para su posterior uso
df_items_Normalizado = json_normalize(df_items['items']) 

In [ ]:
# Quitamos la columna "items" del DataFrame original y combinamos con el df normalizado
df3 = pd.concat([df_items.drop(columns=['items']), df_items_Normalizado], axis=1) 

In [ ]:
# Pasamos el Dataframe normalizado a CSV para su posterior uso
df3.to_csv('user_items_normalizado.csv', index=False) 

In [ ]:
# Creamos una lista vacia donde almacenaremos las filas del archivo json
lista= []

with open(archivo2, encoding='utf-8') as file:
    for line in file.readlines():
        lista.append(ast.literal_eval(line))

# Creamos un DataFrame con la lista recientemente llena 
df  = pd.DataFrame(lista) 

In [ ]:
# Desanidamos la columna "reviews" para su posterior uso
df_reviews = df.explode("reviews") 

In [ ]:
# Reseteamos el indice para evitar problemas a futuro
df_reviews.reset_index(drop=True, inplace=True) 

In [ ]:
# Normalizamos la columna "reviews" para su posterior uso
df_reviews_Normalizado =json_normalize(df_reviews['reviews']) 

In [ ]:
# Quitamos la columna "reviews" del Dataframe original y combinamos con el df normalizado
df_revieww = pd.concat([df_reviews.drop(columns=['reviews']), df_reviews_Normalizado], axis=1) 

In [ ]:
# Quitamos columnas que no nos van a servir
columnas = ["funny", "last_edited","helpful"] 
df_review = df_revieww.drop(columns=columnas) 

In [ ]:
# Pasamos el Dataframe normalizado a CSV para su posterior uso
df_review.to_csv('user_reviews_normalizado.csv', index=False) 

In [ ]:
# Creamos una lista vacia donde almacenaremos las filas del archivo json
lista = []
with gzip.open(archivo3, 'rt', encoding='utf-8') as file:
    for line in file:
        lista.append(js.loads(line))


# Creamos un DataFrame con la lista recientemente llena 
df = pd.DataFrame(lista)

In [ ]:
# Seleccinamos solamente los no nulos
df_games = df.iloc[88310:]

In [ ]:
# Ordenamos el Dataframe
df_ordenado = ["id","title","app_name","developer","genres","specs","tags","price","publisher","url","release_date","reviews_url","early_access"]

df2 = df_games[df_ordenado]

In [ ]:
# Rellenamos valores faltantes de la columna "app_name" con la columna "title"
df2['app_name'] = df2['app_name'].combine_first(df2['title']) 

# Rellenamos valores nulos de la columna "developer" con la columna "publisher" 
df2['developer'].fillna(df2['publisher'], inplace=True)  

# Rellenamos valores nulos de la columna "genres" con la columna "tags"
df2['genres'].fillna(df2['tags'], inplace=True) 
df2['tags'].fillna(df2['genres'], inplace=True)

# Convertimos la columna a string
df2['genres'] = df2['genres'].astype(str)

# Eliminamos "Free to Play" y los caracteres no deseados de la columna "genres"
# Eliminamos todas las ocurrencias de "Free to Play" y formatear la columna "genres"
df2['genres'] = df2['genres'].str.replace("'Free to Play',", "", regex=False)
df2['genres'] = df2['genres'].str.replace(", 'Free to Play'", "", regex=False)
df2['genres'] = df2['genres'].str.replace("'Free to Play'", "", regex=False)

# Eliminamos los corchetes, comillas simples y comillas dobles, y espacios adicionales
df2['genres'] = df2['genres'].str.replace("[", "", regex=False)\
                             .str.replace("]", "", regex=False)\
                             .str.replace("'", "", regex=False)\
                             .str.replace('"', "", regex=False)\
                             .str.strip()

# Separar los géneros por coma y tomar el primer valor
df2['genres'] = df2['genres'].str.split(',').str[0]


 # Convertimos el tipo de dato de esta columna para poder hacer modificaciones
df2['price'] = df2['price'].astype(str)

# Reemplazamos todos los valores que significan lo mismo por un unico valor "Free to Play"
df2['price'] = df2['price'].replace(['Free','Play for Free', 'Free to Try', 'Play for Free!', 'Free to Use','Free Demo'], 'Free to Play') 

# Limpiamos el DataFrame
columnas = ["title","specs","publisher","url","reviews_url","early_access"]
df2 = df2.drop(columns=columnas, axis=1)

In [ ]:
# Pasamos el DataFrame normalizado a CSV para su posterior uso
df2.to_csv('steam_games_normalizado.csv', index=False) 